In [ ]:
#%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
#### %run ../Tipping/Group\ ARD.ipynb

### CHANGED STUFF

In [ ]:
def GroupARD(X, Y, w, in_group = 1, alpha_max = 100, max_iterations = 1000, M_zero = "unknown"):

    N = len(X)
    M = len(X[0])
    
    a = np.repeat(1.0, M) # alphas
    b = 1.0               # Beta = 1/sig^2 

    X1 = X

    deletions = []
    old_alphas = [a]

    for ard_iteration in range(max_iterations):

        # Sigma = (b*XTX + A)^-1
        Sigma = np.linalg.inv(b*np.dot(X1.T,X1) + np.diag(a))

        # mu = b*Sigma*X.T*Y
        mu = b*np.dot(np.dot(Sigma, X1.T), Y)

        gamma = 1.0 - a*np.diag(Sigma)
        group_gamma = np.array([np.sum(gamma[i:i+in_group]) for i in range(0, len(gamma), in_group)])
        mu_squared = mu**2
        group_mu = np.array([np.sum(mu_squared[i:i+in_group]) for i in range(0, len(mu_squared), in_group)])
        a_new = group_gamma/group_mu

        error = np.sum((Y - np.dot(X1, mu))**2)
        b_new = (N - np.sum(gamma))/error

        a = [alpha for alpha in a_new for k in range(in_group)]
        b = b_new

        print("\nIteration: ", ard_iteration, " beta = ", b, " Squared-Error = ", error)  

        over = [i for i in range(len(a))if a[i] > alpha_max]
        if over:
            print("Deletions: ", len(over))
            deletions = [over] + deletions
            X1 = np.delete(X1,over,axis=1)
            a = np.delete(a,over)
        else:
            a_converge = np.sum((a - np.array(old_alphas[-1]))**2)
            print("Alpha distance = ", a_converge, "   Max alpha = ", np.max(a))
            if a_converge < .00001:
                break
        old_alphas.append(a)


    # Recover mu
    for i in deletions:
        for j in i:
            a = np.insert(a,j,-1)
            mu = np.insert(mu,j,0)

    df = pd.DataFrame(list(zip(a, mu, w)), columns = ['alpha', 'mu', 'w'])
    print("\n", df)
    print("\nDeletions:", np.sum([len(i) for i in deletions])/in_group, "out of ", M_zero)
    
    return mu

In [ ]:
### Initialize values
t_max = 250      # length of the simulation
num_neurons = 5  # number of neurons being simulated
num_timelag = 5  # number of time steps back in time are weighted 
sigma = 0.005       # Choose sigma value for the Gaussian noise

# initialize activity of all neurons to 1.0 for first 'num_timelag' time steps 
# (could use random start instead of 1)
activity = np.ones((num_neurons, num_timelag))
# activity = np.random.uniform(0,1,(num_neurons, num_timelag))

# make space for future activity data, 't_max' time steps all initialized to 0
activity = np.hstack((activity, np.zeros((num_neurons, t_max))))

In [ ]:
# def add_ring(size, diag_dists, diag_vals): # size = N neurons, diag_indexs = array of diagonal indices
#     a = np.zeros((size,size))
#     for dist, val in zip(diag_dists, diag_vals):
#         if 2*dist > size: # Also watch out for overlapping dists, e.g. 2 and 4 or 3 and 6, etc.
#             print("Diagonal value " + dist + " is too big")
#             return -1
#         for j in range(size):
#             a[j][(j+dist)%size] = val
#             a[(j+dist)%size][j] = val
#     return a

# def add_timelag(m, num_timelag, decay): # m is 2D array from add_diag, num_timelag is depth dimension
#     a = m
#     for i in range(num_timelag)[1:]:
#         a = np.dstack((a,m*decay**i))
#     return a

# weights = add_ring(num_neurons, [0,1,2], [0.4,0.2,-.6])
# weights = add_timelag(weights, num_timelag, 0.4)

# activity[1:,num_timelag-1] = 0



################################################################
### PICK RING ABOVE OR RANDOM BELOW
################################################################

### Pick weights, normally distributed
weights = np.random.normal(0,1.0,(num_neurons, num_neurons, num_timelag))/(num_neurons*num_timelag)

## Modify 'weights' such that individual neruons get 90% of the activity they had at the previous timestep
## while the rest of their own history is weighted at 0
for i in range(num_neurons):
    #weights[i][i] = 0.0
    weights[i][(i+1)%num_neurons] = 0.0
    weights[i][i][-1] = 0.90

In [ ]:
### Simulate neuron activity data, update 'activity'
for t in range(t_max):
    for i in range(num_neurons):
        
        activity_ti = np.ravel(activity[:,t:t+num_timelag]) @ np.ravel(weights[i])
        activity_ti += 0 if sigma <= 0 else np.random.normal(0,sigma) # Add Gaussian noise
        activity[i,t+num_timelag] = activity_ti

In [ ]:
### Formulate M, a matrix of the relevant activities of each neuron at every time point
m = np.zeros((t_max, num_neurons*num_timelag))
for i in range(t_max):
    for j in range(num_neurons):
        m[i][j*num_timelag:(j+1)*num_timelag] = activity[j,i:i+num_timelag]
        
### Redefine actvity to ignore first 'num_timelag' steps of initialized activity
new_activity = activity[:,num_timelag:].T

In [ ]:
### Visualize the actual activity and recovered activity of a neuron
neuron_n = [0,1] # List of neurons to be observed
weights = np.reshape(weights, (num_neurons,num_neurons*num_timelag), order = 'C').T

f ,ax = plt.subplots(len(neuron_n),3, figsize=(15,5*len(neuron_n)))
for i,j in enumerate(neuron_n):
    ax[i][0].plot(new_activity.T[j]) # Actual activity
    ax[i][1].plot((m @ weights).T[j]) # Recovered Activity Using Real Weights
    ax[i][2].plot((m @ weights).T[j] - new_activity.T[j]) # Error
    ax[i][0].set_ylabel(r'Neuron ' + str(i))
    
for axis, title in zip(ax[0], [r'Actual Activity', r'Recovered Activity w/ Real Weights', r'Error']):
    axis.set_title(title)
    
plt.show()

In [ ]:
### Calculate best estimate of weights using maximum likelihood
### IMPORTANT: We caclulate the pseudoinverse directly instead of doing " inv(m.T @ m) @ m.T "
### For some reason this gives good answers, presumably problems with inverse stability screw up the other way
w_est = np.linalg.solve(m.T @ m , m.T @ new_activity)

In [ ]:
### Visualize the actual activity and recovered activity of a neuron
neuron_n = [0,1]  # List of neurons to be observed

f2 ,ax2 = plt.subplots(len(neuron_n),3, figsize=(15,5*len(neuron_n)))
for i,j in enumerate(neuron_n):
    ax2[i][0].plot(new_activity.T[j]) # Actual activity
    ax2[i][1].plot((m @ w_est).T[j]) # Recovered Activity using Estimated Weights
    ax2[i][2].plot((m @ w_est).T[j] - new_activity.T[j]) # Error
    ax2[i][0].set_ylabel(r'Neuron ' + str(i))
    
for axis, title in zip(ax2[0], [r'Actual Activity', r'Recovered Activity w/ Estimated Weights', r'Error']):
    axis.set_title(title)

In [ ]:
mu = GroupARD(m, new_activity[:,0], weights[:,0], alpha_max=1000000, in_group=num_timelag)

In [ ]:
print(np.shape(m), np.shape(new_activity[:,0]), np.shape(weights[:,0]))

In [ ]:
f = plt.figure()
ax = f.add_subplot(111)

plt.scatter(np.arange(0.5,25.5,1),weights[:,0], color = 'blue', label='Actual Weights')
plt.scatter(np.arange(0.5,25.5,1),w_est[:,0], color = 'green', label='Linear Regression')
plt.scatter(np.arange(0.5,25.5,1),mu, color = 'red', label='Group ARD')
plt.xlim([0,25])
plt.ylim([-0.2,1])
plt.ylabel(r'Weight')
plt.xlabel(r'Source of Connections')
plt.title(r'Neuron 1 Connection Weights')
plt.legend()

for i in np.arange(5,25,5):
    plt.axvline(x=i, linewidth=2, color='gray')
    
ax.set_xticks([2.5,7.5,12.5,17.5,22.5])
ax.set_xticklabels(['Neuron 1','Neuron 2','Neuron 3','Neuron 4','Neuron 5',])
plt.tight_layout()

In [ ]:
plt.show()